In [1]:
from fastai.basic_train import LearnerCallback

In [2]:
from fastai.callbacks.tensorboard import GANTensorboardWriter

In [1]:
import os
import sys
from pathlib import Path
PATH = os.path.join(os.getcwd(), '..', '..')
sys.path.append(PATH)

import torch
import numpy as np
import matplotlib.pyplot as plt

from fastai.vision import *

from src.model_utils.cyclegan import CycleGAN
from src.model_utils.databunch import ImageTupleList
from src.model_utils.callbacks import CycleGANTrainer
from src.model_utils.loss import CycleGanLoss

In [2]:
import os
from pathlib import Path
data_dir = Path('../../data/easy')

In [3]:
data = (ImageTupleList.from_folders(data_dir, 'trainA', 'trainB')
                          .split_none()
                          .label_empty()
                          .transform(get_transforms(), size = 56)
                          .databunch(bs = 4))
cycle_gan = CycleGAN(ch_in = 3, ch_out = 3, 
                     disc_layers = 3, 
                     gen_blocks = 5)
learn = Learner(data, cycle_gan, 
                loss_func = CycleGanLoss(cycle_gan), 
                opt_func = partial(optim.Adam, betas = (0.5, 0.99)),
                callback_fns = [CycleGANTrainer])

In [4]:
learn.fit(1)

epoch,train_loss,valid_loss,id_loss,gen_loss,cyc_loss,D_A_loss,D_B_loss,time


> /userhome/34/h3509807/fastai/CycleGAN-MultiMNIST/src/model_utils/callbacks.py(48)on_batch_end()
-> self.G_A.zero_grad(); self.G_B.zero_grad()
(Pdb) l
 43  	        self.gen_smter.add_value(self.loss_func.gen_loss.detach().cpu())
 44  	        self.cyc_smter.add_value(self.loss_func.cyc_loss.detach().cpu())
 45  	
 46  	    def on_batch_end(self, last_input, last_output, **kwargs):
 47  	        set_trace()
 48  ->	        self.G_A.zero_grad(); self.G_B.zero_grad()
 49  	        fake_A, fake_B = last_output[0].detach(), last_output[1].detach()
 50  	        real_A, real_B = last_input
 51  	        self._set_trainable(D_A=True)
 52  	        self.D_A.zero_grad()
 53  	        loss_D_A = 0.5 * (self.crit(self.D_A(real_A), True) + self.crit(self.D_A(fake_A), False))
(Pdb) u
> /userhome/31/h3509807/anaconda3/envs/fastai-2020/lib/python3.6/site-packages/fastai/callback.py(241)_call_and_update()
-> new = ifnone(getattr(cb, f'on_{cb_name}')(**self.state_dict, **kwargs), dict())
(Pdb) l
236 

(Pdb) type(cb)
<class 'src.model_utils.callbacks.CycleGANTrainer'>
(Pdb) l
247  	    def __call__(self, cb_name, call_mets=True, **kwargs)->None:
248  	        "Call through to all of the `CallbakHandler` functions."
249  	        if call_mets:
250  	            for met in self.metrics: self._call_and_update(met, cb_name, **kwargs)
251  	        for cb in self.callbacks: self._call_and_update(cb, cb_name, **kwargs)
252  	
253  	    def set_dl(self, dl:DataLoader):
254  	        "Set the current `dl` used."
255  	        if hasattr(self, 'cb_dl'): self.callbacks.remove(self.cb_dl)
256  	        if isinstance(dl.dataset, Callback):
257  	            self.callbacks.append(dl.dataset)
(Pdb) d
> /userhome/34/h3509807/fastai/CycleGAN-MultiMNIST/src/model_utils/callbacks.py(48)on_batch_end()
-> self.G_A.zero_grad(); self.G_B.zero_grad()
(Pdb) u
> /userhome/31/h3509807/anaconda3/envs/fastai-2020/lib/python3.6/site-packages/fastai/callback.py(241)_call_and_update()
-> new = ifnone(getattr(cb, f

(Pdb) type(self.state_dict)
<class 'dict'>
(Pdb) self.state_dict.keys()
dict_keys(['epoch', 'iteration', 'num_batch', 'skip_validate', 'n_epochs', 'pbar', 'metrics', 'stop_training', 'last_input', 'last_target', 'train', 'stop_epoch', 'skip_step', 'skip_zero', 'skip_bwd', 'last_output', 'last_loss', 'smooth_loss'])
(Pdb) last_input.shape
*** NameError: name 'last_input' is not defined
(Pdb) self.state_dict['last_input'].shape
*** AttributeError: 'list' object has no attribute 'shape'
(Pdb) type(self.state_dict['last_input'])
<class 'list'>
(Pdb) self.state_dict['last_input'][0].shape
torch.Size([4, 3, 56, 56])
(Pdb) type(self.state_dict['last_output'])
<class 'list'>
(Pdb) self.state_dict['last_output'][0].shape
torch.Size([4, 3, 56, 56])
(Pdb) self.__class__
<class 'fastai.callback.CallbackHandler'>
(Pdb) l
247  	    def __call__(self, cb_name, call_mets=True, **kwargs)->None:
248  	        "Call through to all of the `CallbakHandler` functions."
249  	        if call_mets:
250  	    

(Pdb) cb_name
'batch_end'
(Pdb) l
246  	
247  	    def __call__(self, cb_name, call_mets=True, **kwargs)->None:
248  	        "Call through to all of the `CallbakHandler` functions."
249  	        if call_mets:
250  	            for met in self.metrics: self._call_and_update(met, cb_name, **kwargs)
251  ->	        for cb in self.callbacks: self._call_and_update(cb, cb_name, **kwargs)
252  	
253  	    def set_dl(self, dl:DataLoader):
254  	        "Set the current `dl` used."
255  	        if hasattr(self, 'cb_dl'): self.callbacks.remove(self.cb_dl)
256  	        if isinstance(dl.dataset, Callback):
(Pdb) y
*** NameError: name 'y' is not defined
(Pdb) u
> /userhome/31/h3509807/anaconda3/envs/fastai-2020/lib/python3.6/site-packages/fastai/callback.py(308)on_batch_end()
-> self('batch_end', call_mets = not self.state_dict['train'])
(Pdb) d
> /userhome/31/h3509807/anaconda3/envs/fastai-2020/lib/python3.6/site-packages/fastai/callback.py(251)__call__()
-> for cb in self.callbacks: self._cal

BdbQuit: 